In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from src.models.utils.common import get_project_root

ModuleNotFoundError: No module named 'src'

In [ ]:
# Load results
experiment_name = "v1_initial_run"
results_dir = os.path.join(get_project_root(), "results", experiment_name)
results = pd.read_csv(os.path.join(results_dir, "optuna_results.csv"))

In [ ]:
# Filter out overfitted trials s
non_overfitted = results[
    (results["overfit"] < 0.05)
    & (
        results["model"] != "RandomForest"
    )  # Exclude RandomForest due to consistent overfitting (not generalizing well)
    & (
        results["mean_test_precision"] > 0.78
    )  # Marketing: minimize false positives
].copy()
print(f"\nNumber of non-overfitted trials: {len(non_overfitted)}")

print(non_overfitted["model"].value_counts())

# Calculate robust F1
non_overfitted["robust_f1"] = (
    non_overfitted["mean_test_f1"] - 0.3 * non_overfitted["overfit"]
)


In [ ]:
# Top 3 per model by robust F1
top_models = []
for model in non_overfitted["model"].unique():
    model_top = non_overfitted[non_overfitted["model"] == model].nlargest(
        3, "robust_f1"
    )
    top_models.append(model_top)
top_models = pd.concat(top_models).reset_index(drop=True)

In [ ]:
# Parse params for readability

params_df = pd.json_normalize(top_models["params"])
top_models = pd.concat([top_models.drop(columns="params"), params_df], axis=1)

In [ ]:
# Build summary table
summary_cols = [
    "model",
    "mean_test_f1",
    "mean_test_accuracy",
    "mean_test_precision",
    "mean_test_recall",
    "overfit",
    "params",
]

non_overfitted.groupby("model")[summary_cols].describe().T

In [ ]:
# Scatterplot: Robust F1 vs Precision

sns.scatterplot(
    data=top_models,
    x="robust_f1",
    y="mean_test_precision",
    hue="model",
    size="mean_test_accuracy",
    sizes=(50, 200),
)
plt.axhline(0.78, color="red", linestyle="--", label="Precision Threshold (business)")
plt.xlabel("Robust F1 (F1 - Overfit)")
plt.ylabel("Test Precision")
plt.title("Top Models: Robust F1 vs Precision")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


Model Selection
- Non-overfitted trials: {len(non_overfitted)} (GradientBoosting/XGBoost, precision > 0.8).
- Top models: {len(top_models)} (3 per model by robust F1).
- Selected for refinement: 3 (2 XGBoost, 1 GradientBoosting).
- Criteria: High robust F1, precision > 0.8 for marketing.
- Recommendation: XGBoost leads; GradientBoosting competitive, included for refinement.
- Next: Hyperparameter analysis in `hyperparameter_analysis.ipynb`.

In [ ]:
# Select top models for refinement
selected_models = pd.concat(
    [
        top_models[top_models["model"] == "XGBoost"].nlargest(2, "robust_f1"),
        top_models[top_models["model"] == "GradientBoosting"].nlargest(1, "robust_f1"),
    ]
).reset_index(drop=True)